### **Bioinformatics Project - Computational Drug Discovery [Part 3] Descriptor Calculation and Dataset Preparation**

In Part 3, we will be calculating molecular descriptors that are essentially quantitative description of the compounds in the dataset. Finally, we will be preparing this into a dataset for subsequent model building in Part 4.

### **Load bioactivity data**
Download the curated ChEMBL bioactivity data that has been pre-processed from Parts 1 and 2 of this Bioinformatics Project series. Here we will be using the **acetylcholinesterase_bioactivity_data_with_descipator.csv** file that essentially contain the pIC50 values that we will be using for building a regression model.

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/acetylcholinesterase_bioactivity_data_with_descriptors.csv')
df.sample(3)

molecule_chembl_id     class  \
2673      CHEMBL3632988  inactive   
1123         CHEMBL1678    active   
90          CHEMBL27675    active   

                                       canonical_smiles       MW     LogP  \
2673  Cc1c(C)c2c(c(C)c1O)CCC(C)(C(=O)NCCNc1c3c(nc4cc...  508.062  5.70986   
1123      COc1cc2c(cc1OC)C(=O)C(CC1CCN(Cc3ccccc3)CC1)C2  379.500  4.36110   
90                    C[n+]1ccn(COCCS(C)(=O)=O)c1/C=N/O  262.311 -0.86060   

      NumHDonors  NumHAcceptors     pIC50  
2673         3.0            5.0  6.677781  
1123         0.0            4.0  7.935542  
90           1.0            6.0  4.576754

In [3]:
df.shape

(2876, 8)

In [4]:
selection = ['canonical_smiles','molecule_chembl_id']
df_select = df[selection]
df_select.head(3)

canonical_smiles molecule_chembl_id
0              CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1       CHEMBL133897
1         O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1       CHEMBL336398
2  CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1       CHEMBL131588

In [5]:
df_select.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [6]:
# Sample of data in bash
! cat molecule.smi | head -5

CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1	CHEMBL133897
O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1	CHEMBL336398
CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1	CHEMBL131588
O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F	CHEMBL130628
CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C	CHEMBL130478


In [7]:
# Total number of rows shows in bash
! cat molecule.smi | wc -l

2876


### **Calculate fingerprint descriptors**
### **Download PaDEL-Descriptor**


In [8]:
! pip install padelpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 69.8 MB/s eta 0:00:00


### **Prepare fingerprint XML**
### **Download fingerprint XML files**


In [9]:
! wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
! unzip fingerprints_xml.zip

--2023-04-21 14:32:18--  https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip [following]
--2023-04-21 14:32:18--  https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0s      

2023-04-21 14:32:19 (66.7 MB/s) - ‘fingerprints_xml.zip’ saved [10871/10871]

Archive:  fingerprints_xm

### **List and sort fingerprint XML files**

In [10]:
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

['AtomPairs2DFingerprintCount.xml',
 'AtomPairs2DFingerprinter.xml',
 'EStateFingerprinter.xml',
 'ExtendedFingerprinter.xml',
 'Fingerprinter.xml',
 'GraphOnlyFingerprinter.xml',
 'KlekotaRothFingerprintCount.xml',
 'KlekotaRothFingerprinter.xml',
 'MACCSFingerprinter.xml',
 'PubchemFingerprinter.xml',
 'SubstructureFingerprintCount.xml',
 'SubstructureFingerprinter.xml']

In [11]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

### **Create a dictionary**

In [12]:
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

In [13]:
fp['AtomPairs2D'] # Dictonary key

'AtomPairs2DFingerprinter.xml'

### **Calculate PaDEL descriptors**
There are 12 fingerprint types in PaDEL. To calculate all 12, make sure to make adjustments to the *descriptortypes* input argument to any of the ones in the *fp* dictionary variable as shown above, e.g. SubstructureFingerprintCount.xml

In [14]:
fp

{'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'MACCS': 'MACCSFingerprinter.xml',
 'PubChem': 'PubchemFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml',
 'Substructure': 'SubstructureFingerprinter.xml'}

In [15]:
fp['PubChem'] # dictionary  key for pubchem

'PubchemFingerprinter.xml'

In [16]:
from padelpy import padeldescriptor

fingerprint = 'PubChem'

fingerprint_output_file = ''.join([fingerprint,'.csv']) #PubChem.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='molecule.smi', 
                d_file=fingerprint_output_file, #'PubChem.csv'
                #descriptortypes='PubChemFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

### **Display calculated fingerprints**

In [17]:
descriptors = pd.read_csv(fingerprint_output_file)
descriptors

Name  PubchemFP0  PubchemFP1  PubchemFP2  PubchemFP3  \
0      CHEMBL133897           1           1           1           0   
1      CHEMBL336398           1           1           1           0   
2      CHEMBL131588           1           1           0           0   
3      CHEMBL130628           1           1           1           0   
4      CHEMBL130478           1           1           0           0   
...             ...         ...         ...         ...         ...   
2871  CHEMBL3972214           1           1           1           0   
2872  CHEMBL3910142           0           0           0           0   
2873     CHEMBL8706           1           1           0           0   
2874      CHEMBL972           1           1           1           0   
2875  CHEMBL3291019           0           0           0           0   

      PubchemFP4  PubchemFP5  PubchemFP6  PubchemFP7  PubchemFP8  ...  \
0              0           0           0           0           0  ...   
1              0           0           0           0           0  ...   
2              0           0           0           0           0  ...   
3              0           0           0           0           0  ...   
4              0           0           0           0           0  ...   
...          ...         ...         ...         ...         ...  ...   
2871           0           0           0           0           0  ...   
2872           0           0           0           0           0  ...   
2873           0           0           0           0           0  ...   
2874           0           0           0           0           0  ...   
2875           0           0           0           0           0  ...   

      PubchemFP871  PubchemFP872  PubchemFP873  PubchemFP874  PubchemFP875  \
0                0             0             0             0             0   
1                0             0             0             0             0   
2                0             0             0             0             0   
3                0             0             0             0             0   
4                0             0             0             0             0   
...            ...           ...           ...           ...           ...   
2871             0             0             0             0             0   
2872             0             0             0             0             0   
2873             0             0             0             0             0   
2874             0             0             0             0             0   
2875             0             0             0             0             0   

      PubchemFP876  PubchemFP877  PubchemFP878  PubchemFP879  PubchemFP880  
0                0             0             0             0             0  
1                0             0             0             0             0  
2                0             0             0             0             0  
3                0             0             0             0             0  
4                0             0             0             0             0  
...            ...           ...           ...           ...           ...  
2871             0             0             0             0             0  
2872             0             0             0             0             0  
2873             0             0             0             0             0  
2874             0             0             0             0             0  
2875             0             0             0             0             0  

[2876 rows x 882 columns]

### **Preparing the X and Y Data Matrices**

In [18]:
x = descriptors.drop('Name', axis=1)
y = df['pIC50']
x.shape,y.shape

((2876, 881), (2876,))

#### **Combining X and Y variable**

In [19]:
df2 = pd.concat([x,y], axis=1)
df2

PubchemFP0  PubchemFP1  PubchemFP2  PubchemFP3  PubchemFP4  PubchemFP5  \
0              1           1           1           0           0           0   
1              1           1           1           0           0           0   
2              1           1           0           0           0           0   
3              1           1           1           0           0           0   
4              1           1           0           0           0           0   
...          ...         ...         ...         ...         ...         ...   
2871           1           1           1           0           0           0   
2872           0           0           0           0           0           0   
2873           1           1           0           0           0           0   
2874           1           1           1           0           0           0   
2875           0           0           0           0           0           0   

      PubchemFP6  PubchemFP7  PubchemFP8  PubchemFP9  ...  PubchemFP872  \
0              0           0           0           1  ...             0   
1              0           0           0           1  ...             0   
2              0           0           0           1  ...             0   
3              0           0           0           1  ...             0   
4              0           0           0           1  ...             0   
...          ...         ...         ...         ...  ...           ...   
2871           0           0           0           1  ...             0   
2872           0           0           0           1  ...             0   
2873           0           0           0           1  ...             0   
2874           0           0           0           1  ...             0   
2875           0           0           0           1  ...             0   

      PubchemFP873  PubchemFP874  PubchemFP875  PubchemFP876  PubchemFP877  \
0                0             0             0             0             0   
1                0             0             0             0             0   
2                0             0             0             0             0   
3                0             0             0             0             0   
4                0             0             0             0             0   
...            ...           ...           ...           ...           ...   
2871             0             0             0             0             0   
2872             0             0             0             0             0   
2873             0             0             0             0             0   
2874             0             0             0             0             0   
2875             0             0             0             0             0   

      PubchemFP878  PubchemFP879  PubchemFP880     pIC50  
0                0             0             0  6.124939  
1                0             0             0  7.000000  
2                0             0             0  4.301030  
3                0             0             0  6.522879  
4                0             0             0  6.096910  
...            ...           ...           ...       ...  
2871             0             0             0  5.481486  
2872             0             0             0  7.537602  
2873             0             0             0  3.301030  
2874             0             0             0  3.301030  
2875             0             0             0  6.455932  

[2876 rows x 882 columns]

In [20]:
df2.to_csv('acetylcholinesterase_bioactivity_data_with_pubchem_fingerprint.csv',index=False)